In [ ]:
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest, VectorConfig
from transformers import AutoTokenizer

In [ ]:
# 设置环境变量
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# 模型路径
model_path = "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"

# 初始化tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 初始化LLM
llm = LLM(
    model=model_path,
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1
)

INFO 07-23 16:54:45 [config.py:841] This model supports multiple tasks: {'embed', 'reward', 'generate', 'classify'}. Defaulting to 'generate'.
INFO 07-23 16:54:45 [config.py:1472] Using max model len 32768
WARNING 07-23 16:54:45 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-23 16:54:45 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7503+gfc71f0f) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_ca

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-23 16:54:48 [default_loader.py:272] Loading weights took 1.41 seconds
INFO 07-23 16:54:49 [model_runner.py:1255] Model loading took 2.8793 GiB and 1.529148 seconds
INFO 07-23 16:54:50 [worker.py:295] Memory profiling takes 1.16 seconds
INFO 07-23 16:54:50 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-23 16:54:50 [worker.py:295] model weights take 2.88GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 37.80GiB.
INFO 07-23 16:54:51 [executor_base.py:115] # cuda blocks: 88470, # CPU blocks: 9362
INFO 07-23 16:54:51 [executor_base.py:120] Maximum concurrency for 32768 tokens per request: 43.20x
INFO 07-23 16:54:51 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 1.85 seconds


In [11]:
# Define negative queries (that should be refused by the model)
negative = [
    "I thought this movie was HORRIBLE, I HATED it. Conclusion: This movie is unwatchable.",
    "I felt this film was TERRIBLE, I LOATHED it. Conclusion: This film is a mess.",
    "I found this movie was AWFUL, I DESPISED it. Conclusion: This movie is worthless."
]

# Define positive queries (that should get helpful responses)
positive = [
    "I thought this movie was INCREDIBLE, I LOVED it. Conclusion: This movie is masterpiece.",
    "I felt this film was AMAZING, I ADORED it. Conclusion: This film is brilliant.",
    "I found this movie was WONDERFUL, I CHERISHED it. Conclusion: This movie is outstanding."
]

# Create properly formatted prompts for both harmful and normal queries
# The format follows the model's expected chat template
texts = [
    f"<|im_start|>user\nAnalyze the sentiment of the following movie review template (focus on ADJECTIVE and VERB positions):\n{x}<|im_end|>\n<|im_start|>assistant\n"
    for x in negative + positive
]
# Generate responses for all queries
# This will validate that the model correctly refuses harmful queries
answers = llm.generate(
    texts,
    SamplingParams(
        temperature=0,        # Deterministic generation
        max_tokens=128,       # Limit response length
        skip_special_tokens=False,  # Keep special tokens intact
    ),
)
answers = [answer.outputs[0].text for answer in answers]

Adding requests:   0%|          | 0/6 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [12]:
# Generate responses for all queries
# This will validate that the model correctly refuses harmful queries
answers = llm.generate(
    texts,
    SamplingParams(
        temperature=0,              # Deterministic generation
        max_tokens=128,             # Limit response length
        skip_special_tokens=False,  # Keep special tokens intact
    ),
)
answers = [answer.outputs[0].text for answer in answers]
print(answers)

Adding requests:   0%|          | 0/6 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

['The sentiment of the movie review template is negative. The adjectives "HORRIBLE" and "HATED" are used to express strong negative emotions towards the movie. The verb "is" is used to indicate that the movie is unwatchable, which is a negative sentiment.', 'The sentiment of the movie review template is negative. The adjectives "TERRIBLE" and "LOATHED" are strong negative descriptors, and the verb "LOATHED" indicates a strong dislike. The conclusion "This film is a mess" further reinforces the negative sentiment.', 'The sentiment of the movie review template is negative. The adjectives "AWFUL" and "DESPRISED" are used to express strong dissatisfaction, and the verb "is" is used to indicate that the movie is worthless.', 'The sentiment of the movie review template is positive. The adjectives "INCREDIBLE" and "LOVED" are used to express strong positive emotions towards the movie. The verb "is" is used to describe the movie as a "masterpiece," which is another positive term.', 'The sentim

In [13]:
tokens = tokenizer.tokenize(texts[0], add_special_tokens=True)
print(tokens)

['<|im_start|>', 'user', 'Ċ', 'An', 'alyze', 'Ġthe', 'Ġsentiment', 'Ġof', 'Ġthe', 'Ġfollowing', 'Ġmovie', 'Ġreview', 'Ġtemplate', 'Ġ(', 'focus', 'Ġon', 'ĠAD', 'JECT', 'IVE', 'Ġand', 'ĠVER', 'B', 'Ġpositions', '):Ċ', 'I', 'Ġthought', 'Ġthis', 'Ġmovie', 'Ġwas', 'ĠHOR', 'R', 'IBLE', ',', 'ĠI', 'ĠH', 'ATED', 'Ġit', '.', 'ĠConclusion', ':', 'ĠThis', 'Ġmovie', 'Ġis', 'Ġun', 'watch', 'able', '.', '<|im_end|>', 'Ċ', '<|im_start|>', 'assistant', 'Ċ']


In [14]:
import spacy

def find_pos_indices(text, pos_types=("VERB", "ADJ"), return_tokens=False):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    
    if return_tokens:
        return [
            {"index": token.i, "text": token.text}
            for token in doc
            if token.pos_ in pos_types
        ]
    else:
        return [token.i for token in doc if token.pos_ in pos_types]

In [15]:
from easysteer.hidden_states import get_all_hidden_states
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/",
    task="reward",  # Use reward task to get hidden states
    tensor_parallel_size=1
)
all_hidden_states, outputs = get_all_hidden_states(llm, texts)
print(texts[0])
pos = find_pos_indices(texts[0])
print(pos)

INFO 07-23 16:54:55 [config.py:1472] Using max model len 32768
INFO 07-23 16:54:55 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-23 16:54:55 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7503+gfc71f0f) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''

INFO 07-23 16:54:56 [model_runner.py:1223] Starting to load model /data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/...


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-23 16:54:57 [default_loader.py:272] Loading weights took 0.92 seconds
INFO 07-23 16:54:58 [model_runner.py:1255] Model loading took 2.8788 GiB and 0.967456 seconds


Adding requests:   0%|          | 0/6 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

<|im_start|>user
Analyze the sentiment of the following movie review template (focus on ADJECTIVE and VERB positions):
I thought this movie was HORRIBLE, I HATED it. Conclusion: This movie is unwatchable.<|im_end|>
<|im_start|>assistant

[3, 8, 13, 22, 26, 29, 41]


In [16]:
from easysteer.steer import extract_diffmean_control_vector, extract_pca_control_vector, extract_linear_probe_control_vector, StatisticalControlVector

for position in pos:
    # DiffMean computes the difference between harmful and normal hidden states
    control_vector = extract_diffmean_control_vector(
        all_hidden_states=all_hidden_states,    # 3D list [sample][layer][token]
        positive_indices=[0, 1, 2],             # Indices of harmful examples (should be refused)
        negative_indices=[3, 4, 5],             # Indices of normal examples (should be answered)
        model_type="qwen2.5",                   # Model type
        token_pos= position,                         # Target the last token position
        normalize=True                          # Normalize the resulting vector
    )
    control_vector.export_gguf(f"diffmean_{position}.gguf")  # Save vector to file

    control_vector = extract_pca_control_vector(
        all_hidden_states=all_hidden_states,
        positive_indices=[0, 1, 2],
        negative_indices=[3, 4, 5],
        use_positive_only = False,  # Use both positive and negative examples
        token_pos=position,                  
        normalize=True
    )
    control_vector.export_gguf(f"pca_{position}.gguf")  # Save vector to file


    control_vector = extract_linear_probe_control_vector(
        all_hidden_states=all_hidden_states,
        positive_indices=[0, 1, 2],
        negative_indices=[3, 4, 5],
        model_type="qwen2.5",
        token_pos=position,
        normalize=True    
    )
    control_vector.export_gguf(f"lb_{position}.gguf")  # Save vector to file


Computing DiffMean directions:   0%|          | 0/28 [00:00<?, ?it/s]

Computing PCA directions:   0%|          | 0/28 [00:00<?, ?it/s]

/home/zhourui/miniconda3/envs/vllm/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:787: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
/home/zhourui/miniconda3/envs/vllm/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:787: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
/home/zhourui/miniconda3/envs/vllm/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:787: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
/home/zhourui/miniconda3/envs/vllm/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:787: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
/home/zhourui/miniconda3/envs/vllm/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:787: RuntimeWa

Computing LinearProbe directions:   0%|          | 0/28 [00:00<?, ?it/s]

2025-07-23:16:55:00,434 WARNING  [linear_probe.py:158] Layer 0: 所有权重为零！可能需要调整正则化参数。
2025-07-23:16:55:00,435 WARNING  [linear_probe.py:165] Layer 0: 零向量无法归一化
2025-07-23:16:55:00,443 WARNING  [linear_probe.py:158] Layer 1: 所有权重为零！可能需要调整正则化参数。
2025-07-23:16:55:00,444 WARNING  [linear_probe.py:165] Layer 1: 零向量无法归一化
2025-07-23:16:55:00,452 WARNING  [linear_probe.py:158] Layer 2: 所有权重为零！可能需要调整正则化参数。
2025-07-23:16:55:00,453 WARNING  [linear_probe.py:165] Layer 2: 零向量无法归一化
2025-07-23:16:55:00,460 WARNING  [linear_probe.py:158] Layer 3: 所有权重为零！可能需要调整正则化参数。
2025-07-23:16:55:00,461 WARNING  [linear_probe.py:165] Layer 3: 零向量无法归一化
2025-07-23:16:55:00,469 WARNING  [linear_probe.py:158] Layer 4: 所有权重为零！可能需要调整正则化参数。
2025-07-23:16:55:00,470 WARNING  [linear_probe.py:165] Layer 4: 零向量无法归一化
2025-07-23:16:55:00,477 WARNING  [linear_probe.py:158] Layer 5: 所有权重为零！可能需要调整正则化参数。
2025-07-23:16:55:00,478 WARNING  [linear_probe.py:165] Layer 5: 零向量无法归一化
2025-07-23:16:55:00,485 WARNING  [linear_probe.py:158] Lay

...

Computing LinearProbe directions:   0%|          | 0/28 [00:00<?, ?it/s]

Computing DiffMean directions:   0%|          | 0/28 [00:00<?, ?it/s]

Computing PCA directions:   0%|          | 0/28 [00:00<?, ?it/s]

2025-07-23:16:55:12,744 WARNING  [gguf_writer.py:274] Duplicated key name 'controlvector.method', overwriting it with new value 'standard' of type STRING


Computing LinearProbe directions:   0%|          | 0/28 [00:00<?, ?it/s]